# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.


### Шаг 1. Обзор данных

In [1]:
import pandas as pd # импорт библиотеки pandas

In [2]:
data = pd.read_csv('/datasets/data.csv') # чтение файла с данными и сохранение в data

In [3]:
data.head(10) # получение первых 10 строк таблицы data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Описание данных:

* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` - уровень образования клиента
* `family_status` - семейное положение
* `gender` - пол клиента
* `income_type` - тип занятости
* `debt` - имел ли задолженность по возврату кредитов
* `total_income` - ежемесячный доход
* `purpose` - цель получения кредита

Обнаруженные недостатки:
- в столбце education есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв
- в столбце days_employed указаны отрицательные значения


Посмотрим общую информацию:

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Мы видим наличие двух столбцов(days_employed и total_income), содержащие пропущенные значения.

Посмотрим корректность названий столбцов

In [5]:
data.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

 Столбцы не содержат пробелов, все знаки в одном регистре и на одном языке

Даее проверим величины, хранящиеся в таблице

In [6]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Вывод:
1. В столбце children минимальное и максимальное значение = -1 и 20 соответственное, что является ошибочным значением.
2. Как и было сказано ранее, в столбце days_employed содержатся отрицательные значения.
3. В столбце dob_years всречеается значение равное 0

### Шаг 2.1 Заполнение пропусков

In [7]:
data.isna().sum() # подсчёт пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

найдём все строки с пропусками в столбце total_income и просмотрим первые 10.

In [8]:
display(data[data['total_income'].isna()].head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Пропущенные значения в столбце days_employed соответствуют пропущенным значениям в столбце total_income	
Всего 2174 пропуска, что достаточно много для того, чтобы удалить их.

Проверим, в каких ежемесячных доходах встречаются пропущенные значения 

In [9]:
data.loc[data['total_income'].isna(), 'income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Заменим пустые значения в ежемесячном доходе на медианные по каждому типу занятости.

In [10]:
median_total_income = data.groupby('income_type')['total_income'].median() # медианное значение ежемесячного доходе
print(median_total_income)                                                 # для каждого типа занятости

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64


In [11]:
data.loc[(data['total_income'].isna() == True) & (data['income_type'] == 'сотрудник'), 'total_income'] = median_total_income[6]
data.loc[(data['total_income'].isna() == True) & (data['income_type'] == 'компаньон'), 'total_income'] = median_total_income[3]
data.loc[(data['total_income'].isna() == True) & (data['income_type'] == 'пенсионер'), 'total_income'] = median_total_income[4]
data.loc[(data['total_income'].isna() == True) & (data['income_type'] == 'госслужащий'), 'total_income'] = median_total_income[2]
data.loc[(data['total_income'].isna() == True) & (data['income_type'] == 'предприниматель'), 'total_income'] = median_total_income[5]

Тоже самое сделаем для столбца трудового стажа в днях, но сперва уберем отрицательны значения

In [12]:
data['days_employed'] = data['days_employed'].abs()                          
median_days_employed = data.groupby('income_type')['days_employed'].median() # медианное значение трудового стажа для каждого типа занятости
print(median_days_employed)

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64


In [13]:
data.loc[data['days_employed'].isna(), 'income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

In [14]:
data.loc[(data['days_employed'].isna() == True) & (data['income_type'] == 'сотрудник'), 'days_employed'] = median_days_employed[6]
data.loc[(data['days_employed'].isna() == True) & (data['income_type'] == 'пенсионер'), 'days_employed'] = median_days_employed[4]
data.loc[(data['days_employed'].isna() == True) & (data['income_type'] == 'компаньон'), 'days_employed'] = median_days_employed[3]
data.loc[(data['days_employed'].isna() == True) & (data['income_type'] == 'госслужащий'), 'days_employed'] = median_days_employed[2]
data.loc[(data['days_employed'].isna() == True) & (data['income_type'] == 'предприниматель'), 'days_employed'] = median_days_employed[5]

Проверим результат

In [15]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Шаг 2.2 Проверка данных на аномалии и исправления.

Проверим уникальные значения в столбце children

In [16]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Значение -1 встречается 47 раз, а значение 20 - 76 раз. На мой взгляд данный факт является технической ошибкой.
Заменим значение -1 на 1. Значение 20 заменим на медианное значение всего столбца, так как непонятно, ввели лишний ноль при вводе 2 или же вместо 0 случайно поставили 20

In [17]:
median_children = data['children'].median()
data['children'] = data['children'].replace(-1, 1)
data['children'] = data['children'].replace(20, median_children,)

In [18]:
data['children'].value_counts()

0.0    14225
1.0     4865
2.0     2055
3.0      330
4.0       41
5.0        9
Name: children, dtype: int64

Определим, какой тип занятость имеют люди с ошибочным нулевым возрастом

In [19]:
data.loc[data['dob_years'] == 0, 'income_type'].value_counts()

сотрудник      55
компаньон      20
пенсионер      20
госслужащий     6
Name: income_type, dtype: int64

Найдем среднее значения возрастов клиентов для каждого типа занятости

In [20]:
mean_dob_years = data.groupby('income_type')['dob_years'].mean()
print(mean_dob_years)

income_type
безработный        38.000000
в декрете          39.000000
госслужащий        40.636737
компаньон          39.697542
пенсионер          59.063019
предприниматель    42.500000
сотрудник          39.821027
студент            22.000000
Name: dob_years, dtype: float64


Заменим ошибочный возраст на среднее значение по каждому типу занятости

In [21]:
data.loc[(data['dob_years'] == 0) & (data['income_type'] == 'сотрудник'), 'dob_years'] = mean_dob_years[6]
data.loc[(data['dob_years'] == 0) & (data['income_type'] == 'пенсионер'), 'dob_years'] = mean_dob_years[4]
data.loc[(data['dob_years'] == 0) & (data['income_type'] == 'компаньон'), 'dob_years'] = mean_dob_years[3]
data.loc[(data['dob_years'] == 0) & (data['income_type'] == 'госслужащий'), 'dob_years'] = mean_dob_years[2]

Проверим результат

In [22]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.00000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.472660,67299.486032,43.49822,0.817236,0.972544,0.080883,1.652253e+05
std,0.750616,139401.804684,12.22943,0.548138,1.420324,0.272661,9.804367e+04
min,0.000000,24.141633,19.00000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,1025.549623,34.00000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,1993.522017,43.00000,1.000000,0.000000,0.000000,1.425944e+05
75%,1.000000,5347.024506,53.00000,1.000000,1.000000,0.000000,1.955499e+05
max,5.000000,401755.400475,75.00000,4.000000,4.000000,1.000000,2.265604e+06


### Шаг 2.3. Изменение типов данных.

Ежемесячный доход и общий трудовой стаж в днях переведем в целочисленные значения

In [23]:
data['total_income'] = data['total_income'].astype('int')
data['days_employed'] = data['days_employed'].astype('int')

Проверим результат

In [24]:
data.dtypes

children            float64
days_employed         int64
dob_years           float64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income          int64
purpose              object
dtype: object

### Шаг 2.4. Удаление дубликатов.

Ранее было установлено, что в столбце 'education' есть одни и те же значения, 
но записанные по-разному: с использованием заглавных и строчных букв

In [25]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Видим наличие дубликатов, отличающихся по регистру. Приведем значения к общему виду

In [26]:
data['education'] = data['education'].str.lower()

In [27]:
data.duplicated().sum() # общее количество дубликатов

71

Удалим их

In [28]:
data = data.drop_duplicates().reset_index(drop = True)


Проверим еще раз

In [29]:
data.duplicated().sum()

0

Возникновение дубликатов могло произойти по причине задвоения данных/ошибочного создания дублей

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим два новых датафрейма, в которых:
каждому уникальному значению из education соответствует уникальное значение education_id — в первом;
каждому уникальному значению из family_status соответствует уникальное значение family_status_id — во втором.

In [30]:
education_dict = data[['education', 'education_id']]
family_status_dict = data[['family_status', 'family_status_id']]

Удалим дубликаты

In [31]:
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)

Удалим из исходного датафрейма столбцы education и family_status, оставив только их идентификаторы: education_id и family_status_id. Новые датафреймы — это те самые «словари», к которым сможно обращаться по идентификатору.

In [32]:
data.drop(columns = ['education', 'family_status'],axis = 1, inplace=True)

Проверим результат:

In [33]:
display(education_dict.head())
display(family_status_dict.head())

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


In [34]:
display(data.head(3))

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1.0,8437,42.0,0,0,F,сотрудник,0,253875,покупка жилья
1,1.0,4024,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0.0,5623,33.0,1,0,M,сотрудник,0,145885,покупка жилья


### Шаг 2.6. Категоризация дохода.

Создадим новый столбец на основании диапазонов ежемесячного дохода

In [35]:
def income(total_income):
    if total_income < 30000:
        return 'E'
    if total_income >= 30001 and total_income <= 50000:
        return 'D'
    if total_income >= 50001 and total_income <= 200000:
        return 'C'
    if total_income >= 200001 and total_income <= 1000000:
        return 'B'
    if total_income > 1000001:
        return 'A'
data['total_income_category'] = data['total_income'].apply(income)    

In [36]:
display(data.head(3))

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1.0,8437,42.0,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1.0,4024,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0.0,5623,33.0,1,0,M,сотрудник,0,145885,покупка жилья,C


### Шаг 2.7. Категоризация целей кредита.

Создадим функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут новые категории. Для начала посмотрим все уникальные значения стобца purpose.

In [37]:
data['purpose'].unique() 

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [38]:
def object(purpose):
    if 'автомобил' in purpose:
        return 'операции с автомобилем'
    if 'недвижимост' in purpose or 'жиль' in purpose:
        return 'операции с недвижимостью'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образован' in purpose:
        return 'получение образования'
data['purpose_category'] = data['purpose'].apply(object)

In [39]:
data.head(3)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1.0,8437,42.0,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1.0,4024,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0.0,5623,33.0,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью


### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

Для дальнейшего ответа на данный вопрос проведем категоризациию столбца 'children', в который войдут новые значения в зависимости от количества детей.

In [40]:
def having_children(children):
    if children == 0:
        return 'бездетные'
    if children > 0 and children <= 2:
        return '1-2 ребенка'
    if children > 2:
        return 'многодетные'
data['children_category'] = data['children'].apply(having_children)

Создадим сводную таблицу с помощью метода pivot_table

In [41]:
children_dobt = data.pivot_table(index='children_category', columns='debt', values='days_employed', aggfunc='count')
children_dobt['ratio'] = children_dobt[1] / children_dobt[0] * 100
display(children_dobt.sort_values(by='ratio', ascending=False))

debt,0,1,ratio
children_category,,,
1-2 ребенка,6268,639,10.194639
многодетные,349,31,8.882521
бездетные,13096,1071,8.178070


##### Вывод 1:

Заемщики, не имеющие детей, более склонны возвращать кредит в срок.

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

Ответим на данный вопрос таким же способом, добавив объединение двух таблиц.

In [42]:
family_debt = data.pivot_table(index='family_status_id', columns='debt', values='days_employed', aggfunc='count') #сводная таблица
family_debt['ratio'] = family_debt[1] / family_debt[0] * 100
family_debt_final = family_debt.merge(family_status_dict, on='family_status_id', how='left') #объединим две таблицы методом merge
display(family_debt_final.sort_values(by='ratio', ascending=False))



,family_status_id,0,1,ratio,family_status
4,4,2536,274,10.804416,Не женат / не замужем
1,1,3763,388,10.310922,гражданский брак
0,0,11408,931,8.160940,женат / замужем
3,3,1110,85,7.657658,в разводе
2,2,896,63,7.031250,вдовец / вдова


##### Вывод 2:

Люди, не состоящие в браке, имеют больший процент невозвратов в срок.  Но те, кто развелись или овдовели, чаще платят в срок, чем люди в браке.

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [43]:
employed_dept = data.pivot_table(index='total_income_category', columns='debt', values='days_employed', aggfunc='count') #сводная таблица
employed_dept['ratio'] = employed_dept[1] / employed_dept[0] * 100
display(employed_dept.sort_values(by='ratio', ascending=False))


debt,0,1,ratio
total_income_category,,,
E,20,2,10.000000
C,14655,1360,9.280109
A,23,2,8.695652
B,4686,356,7.597098
D,329,21,6.382979


##### Вывод 3:  

Люди с самым малым доходом в месяц более склонны к невыплате кредитных платежей в установленные сроки.

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [44]:
purpose_debt = data.pivot_table(index='purpose_category', columns='debt', values='days_employed', aggfunc='count') #сводная таблица
purpose_debt['ratio'] = purpose_debt[1] / purpose_debt[0] * 100
display(purpose_debt.sort_values(by='ratio', ascending=False))

debt,0,1,ratio
purpose_category,,,
операции с автомобилем,3903,403,10.325391
получение образования,3643,370,10.156464
проведение свадьбы,2138,186,8.699719
операции с недвижимостью,10029,782,7.797388


##### Вывод 4:

Наибольшая вероятность стать должником есть у людей, получающих кредитные средства на операции с автомобилем. В то время как кредитные средства на операции с недвижимостью возвращаются в срок чаще остальных.

## Общий вывод:

    
Основываясь на полученных данных можно придти к следующим выводам:
    
1. Заемщики, не имеющие детей, более склонны возвращать кредит в срок.
- Вероятность клиента, принадлежащего к группе "бездетные", стать должником стоставляет 8.1%
- Вероятность клиента, принадлежащего к группе "1-2 ребенка", стать должником стоставляет 10.1% 
- Вероятность клиента, принадлежащего к группе "многодетные", стать должником стоставляет 8.8%

2. Люди, не состоящие в браке, имеют больший процент невозвратов в срок.  В то время как те, кто развелись или овдовели, чаще платят в срок, чем люди в браке. Полученные вероятности невозврата кредита в срок распределились следубщим образом:
- Не женат / не замужем 10.8%
- гражданский брак 10.3%
- женат / замужем 8.1%
- в разводе 7.6%
- вдовец / вдова 7%
   
3. Люди с самым малым доходом в месяц более склонны к невыплате кредитных платежей в установленные сроки. Получившиеся вероятности задолженности по убыванию :

- доход в диапазоне до 30 000 рублей в месяц = 10% (категория E)    
- доход в диапазоне от 50 001 до 200 000 рублей в месяц = 9.2% (категория C)
- доход в диапазоне более 1 000 001 рублей в месяц = 8.6% (категория A)
- доход в диапазоне от 200 001 до 1000 000 рублей в месяц = 7.5% (категория B)
- доход в диапазоне от 30 001 до 50 000 рублей в месяц = 6.3% (категория D)
    
Однако следует учитывать тот факт, что такие категории, как E, A и D, состоят из относительно малого числа данных, в связи с чем, дать обективный вывод затруднительно. 
    
4. Наибольшая вероятность стать должником есть у людей, получающие кредитные средства на операции с автомобилем. В то время как кредитные средства на операции с недвижимостью возвращаются в срок чаще остальных. Получившиеся вероятности задолженности по убыванию :
    
- операции с автомобилем 10.3%
- получение образования 10.1%
- проведение свадьбы 8.6%	
- операции с недвижимостью 7.7% 
 
Подводя итоги можно придти к выводу, что семейное положение и количество детей клиента влияет на факт погашения кредита в срок. Наиболее безопасными для банка заемщиками являются люди без детей, имеющие семейное положение вдова/вдовец.    
    
</div>